In [1]:
import pandas as pd

from experiment.scenario import *
from pathlib import Path
import numpy as np

from experiment.scene_part import ScenePartOBJ, ScenePartTIFF

ImportError: cannot import name 'Scenario' from partially initialized module 'experiment.scenario' (most likely due to a circular import) (C:\Users\Florian\OneDrive - TUM\Universität\24\Master's Thesis\Code\city-to-scan-to-city\experiment\scenario.py)

Simplified example code for demonstration:

<br>

```python
scenario_settings = [
    {"pulse_freq_hz": 150_000,
     "accuracy": 0.02},
    {"pulse_freq_hz": 250_000,
     "accuracy": 0.02},
    {"pulse_freq_hz": 500_000,
     "accuracy": 0.02}
]

e = Experiment(experiment_name, experiment_dirpath, default_config, scenario_settings, scene_parts)

e.setup()

e.setup_surveys()
e.prepare_surveys()
e.run_surveys()

e.setup_reconstructions()
e.prepare_reconstructions()
e.run_reconstructions()

e.evaluate_input_old()
e.evaluate_old()
```

# Experiment Test Case

In [2]:
default_config = scenario_default_config()

input_dirpath = Path(r"C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\01_input")
default_config["input_building_models_filepath"] = str(input_dirpath / "9-276-556.city.json")

scene_parts = [
    {
        "type": "obj",
        "filepath": str(input_dirpath / "9-276-556-LoD22-3D_subset.obj"),
        "up_axis": "z"
    },
    {
        "type": "tif",
        "filepath": str(input_dirpath / "M5_37EN1_5_m_filled_clip_to_subset.TIF"),
        "material_filepath": str(input_dirpath / "M5_37EN1_5_m_filled_clip_to_subset.TIF.mtl"),
        "material_name": "ground"
    }
]

footprint_config = {
    "building_footprints_filepath": r"C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\01_input\delft_subset_footprints.gpkg",
    "building_identifier": "identificatie"
}

experiment_name = "experiment_test_case"
experiment_dirpath = r"C:\Users\Florian\Data\city-to-scan-to-city\Experiments"
scenario_settings = [
    {"pulse_freq_hz": 150_000},
    {"pulse_freq_hz": 250_000},
    {"pulse_freq_hz": 500_000}
]

e = Experiment(experiment_name, experiment_dirpath, default_config, scenario_settings, scene_parts, footprint_config)

In [3]:
e.setup()

In [4]:
# Checking some paths and settings
for name, s in e.scenarios.items():
    print(s.name)
    print(s.survey_config["survey_xml_filepath"])
    print(s.survey_config["survey_output_dirpath"])
    print(s.survey_config["survey_generator_config"]["pulse_freq_hz"])

scenario_000
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_000\scenario_000_survey.xml
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_000\output
150000
scenario_001
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_001\scenario_001_survey.xml
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_001\output
250000
scenario_002
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_002\scenario_002_survey.xml
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\04_survey\scenario_002\output
500000


In [4]:
e.setup_surveys()

Setting up survey for scenario scenario_000 ...

Finished setting up survey for scenario scenario_000 after 0:00:00.012004.

Setting up survey for scenario scenario_001 ...

Finished setting up survey for scenario scenario_001 after 0:00:00.003996.

Setting up survey for scenario scenario_002 ...

Finished setting up survey for scenario scenario_002 after 0:00:00.004004.



In [ ]:
e.prepare_surveys()

In [5]:
e.run_surveys()

Simulating survey for scenario scenario_000 ...

Building survey simulation ...
SimulationBuilder is building simulation ...
SimulationBuilder built simulation in 1.86019810000289 seconds
Starting survey simulation ...
Survey simulation running. Time elapsed: 20 s
Survey simulation has finished after 0:00:20.
Reading the coordinates in which the trajectories are parallel ...
Computing center coordinates between adjacent parallel trajectory coordinates ...
Reading input point clouds and computing within-cloud nearest-neighbor distance ...
Processing swath point cloud 1 of 5 ...
- Processed 355915 points.
Processing swath point cloud 2 of 5 ...
- Processed 357479 points.
Processing swath point cloud 3 of 5 ...
- Processed 359626 points.
Processing swath point cloud 4 of 5 ...
- Processed 356946 points.
Processing swath point cloud 5 of 5 ...
- Processed 355094 points.
Computing mean within-cloud nearest-neighbor distances ...
[0.16373012806816545, 0.17074433351091092, 0.1739623874865185,

In [4]:
e.setup_reconstructions()

Setting up building reconstruction for scenario scenario_000 ...

Finished setting up building reconstruction for scenario scenario_000 after 0:00:00.002009.

Setting up building reconstruction for scenario scenario_001 ...

Finished setting up building reconstruction for scenario scenario_001 after 0:00:00.008000.

Setting up building reconstruction for scenario scenario_002 ...

Finished setting up building reconstruction for scenario scenario_002 after 0:00:00.006559.



In [ ]:
e.prepare_reconstructions()

In [7]:
e.run_reconstructions()

Reconstructing buildings for scenario scenario_000 ...

Starting 3D building reconstruction ...
- Command: geof C:/Users/Florian/Data/city-to-scan-to-city/Experiments/experiment_test_case/05_reconstruction/scenario_000/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/city-to-scan-to-city/Experiments/experiment_test_case/05_reconstruction/scenario_000/config.toml
- Output log file: C:/Users/Florian/Data/city-to-scan-to-city/Experiments/experiment_test_case/05_reconstruction/scenario_000/geoflow_log.txt

Detected environment variable GF_PLUGIN_FOLDER = C:\Program Files\Geoflow\lib\geoflow-plugins
Setting PROJ DATA dir to C:\Program Files\Geoflow\share\proj(default context)
Setting PROJ DATA dir to C:\Program Files\Geoflow\share\proj
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_buildingreconstruction.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_core_io.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_gdal.dll
Loaded C:\Program Files\

In [ ]:
e.setup_evaluations()

In [ ]:
e.run_evaluations()

In [4]:
e.evaluate_input_old()

Input CityJSON: C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\01_input\9-276-556.city.json
CityJSON version = 2.0
EPSG = 7415
bbox = [ 82585.172 445871.250 -24.412 83613.391 446896.812 29.629 ]
=== CityObjects ===
|-- Building (3050)
    |-- BuildingPart (3052)
materials = False
textures = False

Executing FME workspace with command:
C:\Program Files\FME\fme.exe
C:\Users\Florian\OneDrive - TUM\Universität\24\Master's Thesis\Code\city-to-scan-to-city\experiment\computation_area_volume.fmw
--SourceDataset_CITYJSON
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\01_input\9-276-556.city.json
--SourceDataset_CITYJSON_2
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\01_input\9-276-556.city.json
--CITYJSON_IN_LOD_2
1.2
--COORDSYS_1
https://www.opengis.net/def/crs/EPSG/0/7415
--COORDSYS_2
https://www.opengis.net/def/crs/EPSG/0/7415
--COORDSYS_3
https://www.opengis.net/def/crs/EPSG/0/7415
--DestDataset_CITYJS

In [4]:
e.evaluate_old()

In [5]:
for name, s in e.scenarios.items():
    print(f"Evaluating scenario {name} ...")
    print("")
    s.reconstruction.evaluate_old()
    print("")

Evaluating scenario scenario_000 ...

Input CityJSON: C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\05_reconstruction\scenario_000\output\model.json
CityJSON version = 2.0
EPSG = 7415
bbox = [ 83043.727 446327.562 -0.614 83255.305 446496.906 29.601 ]
=== CityObjects ===
|-- Building (90)
    |-- BuildingPart (89)
materials = False
textures = False

Executing FME workspace with command:
C:\Program Files\FME\fme.exe
C:\Users\Florian\OneDrive - TUM\Universität\24\Master's Thesis\Code\city-to-scan-to-city\experiment\computation_area_volume.fmw
--SourceDataset_CITYJSON
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\05_reconstruction\scenario_000\output\model.json
--SourceDataset_CITYJSON_2
C:\Users\Florian\Data\city-to-scan-to-city\Experiments\experiment_test_case\05_reconstruction\scenario_000\output\model.json
--CITYJSON_IN_LOD_2
1.2
--COORDSYS_1
https://www.opengis.net/def/crs/EPSG/0/7415
--COORDSYS_2
https://www.opengis.net/def/

In [4]:
e.run_all()

Executing scenario scenario_000 ...

Building survey simulation ...
SimulationBuilder is building simulation ...
SimulationBuilder built simulation in 4.303864300018176 seconds
Starting survey simulation ...
Survey simulation running. Time elapsed: 11 s

KeyboardInterrupt: 

# Experiment Draft

In [ ]:
experiment_dirpath = ""
input_dirpath = str(Path(experiment_dirpath, "01_input"))
scene_dirpath = str(Path(experiment_dirpath, "02_scene"))
survey_dirpath = str(Path(experiment_dirpath, "03_survey"))
reconstruction_dirpath = str(Path(experiment_dirpath, "04_reconstruction"))
evaluation_dirpath = str(Path(experiment_dirpath, "05_evaluation"))

scene_config = {}

scenario_base_config = {}

independent_variables = {
    "pulse_freq_hz": [],
    "accuracy_m": []
}


# Default Config

Drafting a default config with default values or no values.

In [ ]:
# region Config dictionary

input_dirpath = ""

scene_config = {
    "scene_xml_filepath": "",
    "scene_xml_id": "",
    "scene_name": "",
    "scene_parts": [
        {
            "type": "obj",
            "filepath": str(Path(input_dirpath, "9-276-556-LoD22-3D_subset.obj")),
            "up_axis": "z"
        },
        {
            "type": "tif",
            "filepath": str(Path(input_dirpath, "M5_37EN1_5_m_filled_clip_to_subset.TIF")),
            "material_filepath": str(Path(input_dirpath, "M5_37EN1_5_m_filled_clip_to_subset.TIF.mtl")),
            "material_name": "ground"
        }
    ]
}

survey_generator_config = {
    "survey_template_xml_filepath": "experiment/survey_template.xml",
    "survey_name": "",
    "scene_xml_filepath_with_id": "",  
    "platform_id": "sr22",
    "scanner_id": "riegl_vq-1560i",
    "scanner_settings_id": "scanner_settings",
    "scanner_settings_active": True,
    "pulse_freq_hz": 500_000,
    "scan_angle_deg": 15,
    "scan_freq_hz": 300,
    "detector_settings_accuracy": 0.02,
}

flight_path_config = {
    "flight_path_xml_filepath": "",
    "bbox": [83043, 446327, 83255, 446496],
    "spacing": 40,
    "altitude": 100,
    "velocity": 60,
    "flight_pattern": "parallel",
    "trajectory_time_interval": .05,
    "always_active": False,
    "scanner_settings_id": "scanner_settings"
}

survey_executor_config = {
    "las_output": True,
    "zip_output": True,
    "num_threads": 0,
}

cloud_merge_config = {
    # "clouds_dirpath": "",
    # "output_filepath": ""
}

survey_config = {
    "survey_xml_filepath": "",
    "survey_output_dirpath": "",
    "survey_generator_config": survey_generator_config,
    "flight_path_config": flight_path_config,
    "survey_executor_config": survey_executor_config,
    "cloud_merge_config": cloud_merge_config
}

reconstruction_config = {
    # "config_toml_filepath": "",
    "point_cloud_filepath": "",
    "building_footprints_filepath": str(Path(input_dirpath, "delft_subset_footprints.gpkg")),
    "building_identifier": "fid",
    "reconstruction_output_dirpath": reconstruction_dirpath,
    # "geoflow_log_filepath": str(Path(reconstruction_dirpath, "geoflow_log.txt"))
}

# Final scenario settings

scenario_config = {
    # "building_models_in_filepath": "",
    "crs": "epsg:7415",
    "scene_config": scene_config,
    "survey_config": survey_config,
    "reconstruction_config": reconstruction_config,
}

# endregion

# Scenario Test Case

In [2]:
# region Scenario configuration
experiment_dirpath = r"C:\Users\Florian\Data\city-to-scan-to-city\Scenarios\test_case_01"
input_dirpath = str(Path(experiment_dirpath, "01_input_data"))
survey_dirpath = str(Path(experiment_dirpath, "02_survey"))
reconstruction_dirpath = str(Path(experiment_dirpath, "03_reconstruction"))

scene_config = {
    "scene_xml_filepath": str(Path(survey_dirpath, "Delft_test_case_scene.xml")),
    "scene_xml_id": "delft_test_case",
    "scene_name": "Delft Test Case",
    "scene_parts": [
        {
            "type": "obj",
            "filepath": str(Path(input_dirpath, "9-276-556-LoD22-3D_subset.obj")),
            "up_axis": "z"
        },
        {
            "type": "tif",
            "filepath": str(Path(input_dirpath, "M5_37EN1_5_m_filled_clip_to_subset.TIF")),
            "material_filepath": str(Path(input_dirpath, "M5_37EN1_5_m_filled_clip_to_subset.TIF.mtl")),
            "material_name": "ground"
        }
    ]
}

survey_generator_config = {
    "survey_template_xml_filepath": "experiment/survey_template.xml",
    "survey_name": "Delft_test_case",
    "scene_xml_filepath_with_id": str(Path(survey_dirpath, "Delft_test_case_scene.xml#id_missing")),  
    "platform_id": "sr22",
    "scanner_id": "riegl_vq-1560i",
    "scanner_settings_id": "scanner_settings",
    "scanner_settings_active": True,
    "pulse_freq_hz": 500_000,
    "scan_angle_deg": 15,
    "scan_freq_hz": 300,
    "detector_settings_accuracy": 0.02,
}

flight_path_config = {
    "flight_path_xml_filepath": str(Path(survey_dirpath, "Delft_test_case_flight_path.xml")),
    "bbox": [83043, 446327, 83255, 446496],
    "spacing": 40,
    "altitude": 100,
    "velocity": 60,
    "flight_pattern": "parallel",
    "trajectory_time_interval": .05,
    "always_active": False,
    "scanner_settings_id": "scanner_settings"
}

survey_executor_config = {
    "las_output": True,
    "zip_output": True,
    "num_threads": 0,
}

cloud_merge_config = {
    "clouds_dirpath": "",
    "output_filepath": ""
}

survey_config = {
    "survey_xml_filepath": str(Path(survey_dirpath, "Delft_test_case_survey.xml")),
    "survey_output_dirpath": str(Path(survey_dirpath, "output")),
    "survey_generator_config": survey_generator_config,
    "flight_path_config": flight_path_config,
    "survey_executor_config": survey_executor_config,
    "cloud_merge_config": cloud_merge_config
}

reconstruction_config = {
    # "config_toml_filepath": "",
    "point_cloud_filepath": "",
    "building_footprints_filepath": str(Path(input_dirpath, "delft_subset_footprints.gpkg")),
    "building_identifier": "fid",
    "reconstruction_output_dirpath": reconstruction_dirpath,
    # "geoflow_log_filepath": str(Path(reconstruction_dirpath, "geoflow_log.txt"))
}

# Final scenario settings

scenario_config = {
    # "building_models_in_filepath": "",
    "crs": "epsg:7415",
    "scene_config": scene_config,
    "survey_config": survey_config,
    "reconstruction_config": reconstruction_config,
}

# endregion

In [3]:
s = Scenario(name="Delft_test_case", config=scenario_config)

In [4]:
s.setup_scene()

In [5]:
s.setup_survey()

In [6]:
s.run_survey()

Building survey simulation ...
SimulationBuilder is building simulation ...
SimulationBuilder built simulation in 1.7578828999999132 seconds
Starting survey simulation ...
Survey simulation running. Time elapsed: 88 s
Survey simulation has finished after 0:01:28.
Reading the coordinates in which the trajectories are parallel ...
Computing center coordinates between adjacent parallel trajectory coordinates ...
Reading input point clouds and computing within-cloud nearest-neighbor distance ...
Processing swath point cloud 1 of 5 ...
- Processed 1184161 points.
Processing swath point cloud 2 of 5 ...
- Processed 1189665 points.
Processing swath point cloud 3 of 5 ...
- Processed 1196336 points.
Processing swath point cloud 4 of 5 ...
- Processed 1187598 points.
Processing swath point cloud 5 of 5 ...
- Processed 1181425 points.
Computing mean within-cloud nearest-neighbor distances ...
[0.0969109300788063, 0.09943894488760342, 0.0995335521683156, 0.09696149284360903, 0.09509909708685599]


In [7]:
s.setup_reconstruction()

In [37]:
s.reconstruction.executor.command[1] = s.reconstruction.executor.command[1][1:-1] # f"'{s.reconstruction.executor.command[1]}'"

In [38]:
s.reconstruction.executor.command[5] = s.reconstruction.executor.command[5][1:-1] # f"'{s.reconstruction.executor.command[5]}'"

In [39]:
s.reconstruction.executor.command # = Path(s.reconstruction.executor.command[1]).as_posix()

['geof',
 'C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/reconstruct.json',
 '--verbose',
 '--workdir',
 '--config',
 'C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/config.toml']

In [40]:
s.reconstruction.executor.stdout_log_filepath #= Path(s.reconstruction.executor.stdout_log_filepath).as_posix()

'C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/geoflow_log.txt'

In [46]:
s.reconstruction.cloud_filepath

'C:/Users/Florian/Data/Experiments/test_case_01/02_survey/output/Delft_test_case/2024-05-27_16-02-37/clouds_merged.laz'

In [44]:
s.run_reconstruction()

Starting building reconstruction ...
- Command: geof C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/config.toml
- Output log file: C:/Users/Florian/Data/Experiments/test_case_01/03_reconstruction/geoflow_log.txt

Detected environment variable GF_PLUGIN_FOLDER = C:\Program Files\Geoflow\lib\geoflow-plugins
Setting PROJ DATA dir to C:\Program Files\Geoflow\share\proj(default context)
Setting PROJ DATA dir to C:\Program Files\Geoflow\share\proj
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_buildingreconstruction.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_core_io.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_gdal.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_las.dll
Loaded C:\Program Files\Geoflow\lib\geoflow-plugins\gfp_val3dity.dll
Setting PROJ DATA dir to C:\Program Files\Geoflow\share\proj(default context

# Second version using generator classes plus survey executor

In [2]:
# region Scenario configuration

scene_config = {
    "scene_xml_filepath": str(Path(glb.helios_dirpath, "data/scenes/Delft_scene_generated.xml")),
    "scene_xml_id": "delft",
    "scene_name": "Delft",
    "scene_parts": [
        {
            "type": "obj",
            "filepath": str(Path(glb.helios_dirpath, "data/sceneparts/Delft/9-276-556-LoD22-3D.obj")),
            "up_axis": "z"
        },
        {
            "type": "tif",
            "filepath": str(Path(glb.helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF")),
            "material_filepath": str(Path(glb.helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF.mtl")),
            "material_name": "ground"
        }
    ]
}

survey_generator_config = {
    "survey_template_xml_filepath": "experiment/survey_template.xml",
    "survey_name": "Delft_survey_generated",
    # Since scene xml path would only be known after scene is generated
    # for experiment, perhaps it can't be a global default?
    "scene_xml_filepath_with_id": str(Path(glb.helios_dirpath, "data/scenes/Delft_scene_generated.xml#id_missing")),  
    "platform_id": "sr22",
    "scanner_id": "riegl_vq-1560i",
    "scanner_settings_id": "scanner_settings",
    "scanner_settings_active": True,
    "pulse_freq_hz": 1_000_000,
    "scan_angle_deg": 15,
    "scan_freq_hz": 300,
    "detector_settings_accuracy": 0.02,
}

flight_path_config = {
    "flight_path_xml_filepath": str(Path(glb.helios_dirpath, "data/surveys/Delft_survey_generated_flightpath.xml")),
    "bbox": [82535, 445821, 83663, 446947],
    "spacing": 225.2,
    "altitude": 500,
    "velocity": 60,
    "flight_pattern": "parallel",
    "trajectory_time_interval": .05,
    "always_active": False,
    "scanner_settings_id": "scanner_settings"
}

survey_executor_config = {
    "las_output": True,
    "zip_output": True,
    "num_threads": 0,
}

cloud_merge_config = {
    "clouds_dirpath": "",
    "output_filepath": ""
}

survey_config = {
    "survey_xml_filepath": str(Path(glb.helios_dirpath, "data/surveys/Delft_survey_generated.xml")),
    "output_dirpath": str(Path(glb.helios_dirpath, "output")),
    "survey_generator_config": survey_generator_config,
    "flight_path_config": flight_path_config,
    "survey_executor_config": survey_executor_config,
    "cloud_merge_config": cloud_merge_config
}

reconstruction_config = {
    "config_toml_filepath": "",
    "point_cloud_filepath": "",
    "building_footprints_filepath": "",
    "building_identifier": "",
    "output_dirpath": ""
}

# Final scenario settings

scenario_config = {
    "building_models_in_filepath": "",
    "crs": "epsg:7415",
    "scene_config": scene_config,
    "survey_config": survey_config,
    "reconstruction_config": reconstruction_config,
}

# endregion

In [3]:
s = Scenario(name="Delft_scenario", config=scenario_config)

In [4]:
s.setup_scene()

In [5]:
s.setup_survey()

In [6]:
# s.run()

In [7]:
point_cloud_dirpath = r"C:\Users\Florian\Apps\helios-plusplus-win\output\Delft_survey_generated\2024-05-24_18-32-34"
# point_cloud_filepaths = [str(p) for p in Path(point_cloud_dirpath).iterdir() if p.is_file() and p.suffix == ".laz"]
point_cloud_filepaths = [str(p) for p in Path(point_cloud_dirpath).iterdir() if p.is_file() and str(p).endswith("points.laz")]
# point_cloud_filepaths = point_cloud_filepaths[:2]
s.survey.output_clouds_filepaths = point_cloud_filepaths
point_cloud_filepaths

['C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg000_points.laz',
 'C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg002_points.laz',
 'C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg004_points.laz',
 'C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg006_points.laz',
 'C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg008_points.laz',
 'C:\\Users\\Florian\\Apps\\helios-plusplus-win\\output\\Delft_survey_generated\\2024-05-24_18-32-34\\leg010_points.laz']

In [8]:
s.survey.setup_merger()

In [9]:
s.survey.cloud_merger.get_parallel_trajectory_coords()

Reading the coordinates in which the trajectories are parallel ...


In [10]:
s.survey.cloud_merger.compute_separation_coords()

Computing center coordinates between adjacent parallel trajectory coordinates ...


In [11]:
m = s.survey.cloud_merger

In [12]:
print(m.parallel_trajectory_coords)
print(len(m.parallel_trajectory_coords))
print(m.separation_coords)
print(len(m.separation_coords))

[445821.0, 446046.2, 446271.4, 446496.60000000003, 446721.80000000005, 446947.00000000006]
6
[0, 445933.6, 446158.80000000005, 446384.0, 446609.20000000007, 446834.4, 893668.8]
7


In [13]:
s.survey.cloud_merger.filter_clouds()

Reading input point clouds and computing within-cloud nearest-neighbor distance ...
Processing swath point cloud 1 of 6 ...
- Processed 12677717 points.
Processing swath point cloud 2 of 6 ...
- Processed 13237881 points.
Processing swath point cloud 3 of 6 ...
- Processed 13536285 points.
Processing swath point cloud 4 of 6 ...
- Processed 13560063 points.
Processing swath point cloud 5 of 6 ...
- Processed 13378799 points.
Processing swath point cloud 6 of 6 ...
- Processed 12724182 points.
Computing mean within-cloud nearest-neighbor distances ...
[0.1572372598102812, 0.16199774890980065, 0.16445565481183863, 0.16452550401581273, 0.16359892649224406, 0.15769265338088698]
Computing k-d trees ...
Processing swath point cloud 1 of 6 ...
- has 1 adjacent swath.
- Computing between-cloud nearest-neighbor distance to all adjacent swaths ...
- Filtering points in overlapping area ...
- filter_expression: (Y >= 0 && Y <= 445933.6) || (CC_NNDistance_1 > 0.16199774890980065)
- extra_dims: NND

In [14]:
s.survey.cloud_merger.merge_clouds()

Concatenating filtered point clouds ...
- Total number of points: 71228039
Writing merged point clouds to output location ...
- Written number of points: 71228039


In [ ]:
# s.survey.merge_clouds()

In [17]:
# old_merger = m

# First version of scene, flight path, and survey XML generation

## Scene XML generation

In [1]:
scene_filepath = r"temp\scene.xml"
obj_filepath = Path(glb.helios_dirpath, "data/sceneparts/Delft/9-276-556-LoD22-3D.obj")
tiff_filepath = Path(glb.helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF")
tiff_mat_filepath = Path(glb.helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF.mtl")

NameError: name 'Path' is not defined

In [3]:
scene_part_obj = ScenePartOBJ(obj_filepath, up_axis="z")
scene_part_tiff = ScenePartTIFF(tiff_filepath, tiff_mat_filepath, "ground")

In [4]:
scene = SceneGenerator(xml_id="Delft", name="Delft", filepath=scene_filepath)
scene.add_scene_parts([scene_part_tiff, scene_part_obj])

In [5]:
scene.create_xml_string()
print(scene.xml_string)

<?xml version="1.0" encoding="UTF-8"?>
<document>
    <scene id="Delft" name="Delft">
        
        <part>
            <filter type="geotiffloader">
                <param type="string" key="filepath" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\M5_37EN1_5_m_filled.TIF" />
                <param type="string" key="matfile" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\M5_37EN1_5_m_filled.TIF.mtl" />
                <param type="string" key="matname" value="ground" />
            </filter>
            
        </part>

        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\9-276-556-LoD22-3D.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>
    </scene>
</document>


In [ ]:
scene.write_scene_file()

## Flight path generation

In [6]:
flight_path_filepath = r"temp\flight_path.xml"

def buffer_bbox(bbox, buffer_width = 50):
    bbox_buffer = np.array([-buffer_width, -buffer_width, buffer_width, buffer_width])
    bbox = bbox + bbox_buffer
    width = bbox_float[2] - bbox_float[0]
    height = bbox_float[3] - bbox_float[1]
    print(f"Width:  {width}")
    print(f"Height: {height}")
    return bbox

# bbox as [West, South, East, North]
bbox_float = [82585.1719, 445871.2500, 83613.3906, 446896.7813]
bbox_float = buffer_bbox(bbox_float, buffer_width=50)
bbox_int = np.round(bbox_float).astype(int)

strip_spacing = (bbox_int[3] - bbox_int[1]) / 5
altitude = 500
platform_speed = 60

Width:  1028.2186999999976
Height: 1025.5312999999733


In [7]:
flight_path = FlightPathGenerator(flight_path_filepath, bbox_int, strip_spacing, altitude, platform_speed)

In [8]:
flight_path.compute_waypoints()

In [9]:
flight_path.create_element_tree()

In [9]:
list(flight_path.tree.getroot().iter())

[<Element 'survey' at 0x0000021CA4D9F6F0>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9F240>,
 <Element 'leg' at 0x0000021CA93FA070>,
 <Element 'platformSettings' at 0x0000021CA4D9EC50>,
 <Element 'scannerSettings' at 0x0000021CA4D9EC00>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9E930>,
 <Element 'leg' at 0x0000021CA4D9ED40>,
 <Element 'platformSettings' at 0x0000021CA4D9EBB0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EB10>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9E9D0>,
 <Element 'leg' at 0x0000021CA4D9EDE0>,
 <Element 'platformSettings' at 0x0000021CA4D9E7A0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EA70>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9ECF0>,
 <Element 'leg' at 0x0000021CA4D9ECA0>,
 <Element 'platformSettings' at 0x0000021CA4D9E7F0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EE80>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9F1A0>,
 <

In [10]:
flight_path.create_xml_string()
print(flight_path.xml_string)

<survey>
    <!--leg 000-->
    <leg>
        <platformSettings x="83663.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 001-->
    <leg>
        <platformSettings x="82535.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 002-->
    <leg>
        <platformSettings x="82535.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 003-->
    <leg>
        <platformSettings x="83663.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 004-->
    <leg>
        <platformSettings x="83663.0" y="446271.4" z="500" movePerSec_m="60" />
        <scannerSett

In [11]:
flight_path.write_flight_path_file()

## Survey generation

In [10]:
survey_filepath = r"temp\survey.xml"
platform_xml_path = str(Path(glb.helios_dirpath, "data/platforms.xml#sr22"))
scanner_xml_path = str(Path(glb.helios_dirpath, "data/scanners_als.xml#riegl_vq_780i"))

survey = SurveyGenerator(
    name="Delft",
    filepath=survey_filepath,
    scene=scene,
    platform=platform_xml_path,
    scanner=scanner_xml_path,
    flight_path=flight_path #_filepath
)

In [11]:
survey.create_element_tree()

In [12]:
survey.write_survey_file()

## Generating element tree from scratch

In [5]:
import xml.etree.ElementTree as eT

In [6]:
root = eT.Element("survey")
tree = eT.ElementTree(root)

In [7]:
flight_path.waypoints

array([[ 83663. , 445821. ],
       [ 82535. , 445821. ],
       [ 82535. , 446046.2],
       [ 83663. , 446046.2],
       [ 83663. , 446271.4],
       [ 82535. , 446271.4],
       [ 82535. , 446496.6],
       [ 83663. , 446496.6],
       [ 83663. , 446721.8],
       [ 82535. , 446721.8],
       [ 82535. , 446947. ],
       [ 83663. , 446947. ]])

In [8]:
short_leg_active = "true" if flight_path.always_active else "false"

for i, leg in enumerate(flight_path.waypoints):
    root.append(eT.Comment(f"leg {i:03}"))
    
    leg_element = eT.Element("leg")
    root.append(leg_element)
    
    platform_settings_element = eT.Element(
        "platformSettings",
        attrib={
            "x": str(leg[0]),
            "y": str(leg[1]),
            "z": str(flight_path.altitude),
            "movePerSec_m": str(flight_path.velocity)
        }
    )
    leg_element.append(platform_settings_element)

    scanner_settings_element = eT.Element(
        "scannerSettings",
        attrib={
            "template": str(flight_path.scanner_settings_id),
            "trajectoryTimeInterval_s": str(flight_path.trajectory_time_interval)
        }
    )
    if i % 2 != 0:
        scanner_settings_element.attrib["active"] = str(short_leg_active)
    leg_element.append(scanner_settings_element)    

In [9]:
eT.indent(tree, "    ")

In [20]:
print(eT.tostring(tree.getroot(), encoding="unicode", xml_declaration=False))

<survey>
    <!--leg 000-->
    <leg>
        <platformSettings x="83663.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 001-->
    <leg>
        <platformSettings x="82535.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 002-->
    <leg>
        <platformSettings x="82535.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 003-->
    <leg>
        <platformSettings x="83663.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 004-->
    <leg>
        <platformSettings x="83663.0" y="446271.4" z="500" movePerSec_m="60" />
        <scannerSett

In [29]:
tree.write(r"temp\flight_path_etree.xml", encoding="UTF-8", xml_declaration=False)